In [10]:
import numpy as np
from sklearn.cluster import KMeans
from collections import defaultdict
from math import radians, cos, sin, asin, sqrt
import sqlite3

# -------------------------------
# 📍 Race Locations (lat, lon)
# Add or modify as needed
# -------------------------------
# Connect to the database
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Query to fetch race locations and their lat/lon
query = """
SELECT fc.circuit, fg.latitude, fg.longitude
FROM fone_calendar fc
JOIN fone_geography fg ON fc.geo_id = fg.id
WHERE fc.year = 2025
"""
cursor.execute(query)

# Build the race_locations dictionary
race_locations = {row[0]: (row[1], row[2]) for row in cursor.fetchall()}

# Close the database connection
conn.close()

track_names = list(race_locations.keys())
locations = np.array(list(race_locations.values()))

# -------------------------------
# 🌍 Haversine Distance Function
# -------------------------------
def haversine(loc1, loc2):
    lat1, lon1 = loc1
    lat2, lon2 = loc2
    # convert to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return 6371 * c  # Radius of Earth in km

# -------------------------------
# Step 1: Cluster the Races
# -------------------------------
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(locations)
labels = kmeans.labels_

clusters = defaultdict(list)
for i, label in enumerate(labels):
    clusters[label].append(track_names[i])

# -------------------------------
# Step 2: TSP within Clusters
# -------------------------------
def nearest_neighbor_tsp(track_list):
    unvisited = track_list[:]
    path = [unvisited.pop(0)]
    while unvisited:
        last = path[-1]
        next_track = min(unvisited, key=lambda t: haversine(race_locations[last], race_locations[t]))
        path.append(next_track)
        unvisited.remove(next_track)
    return path

# -------------------------------
# Step 3: Cluster Centroids
# -------------------------------
cluster_centroids = {}
for cluster_id, tracks in clusters.items():
    lats = [race_locations[t][0] for t in tracks]
    lons = [race_locations[t][1] for t in tracks]
    cluster_centroids[cluster_id] = (np.mean(lats), np.mean(lons))

# -------------------------------
# Step 4: TSP over Clusters
# -------------------------------
def tsp_on_centroids(cluster_order):
    unvisited = cluster_order[:]
    path = [unvisited.pop(0)]
    while unvisited:
        last = path[-1]
        next_cluster = min(unvisited, key=lambda c: haversine(cluster_centroids[last], cluster_centroids[c]))
        path.append(next_cluster)
        unvisited.remove(next_cluster)
    return path

cluster_order = tsp_on_centroids(list(clusters.keys()))

# -------------------------------
# Step 5: Flatten Final Schedule
# -------------------------------
final_schedule = []
for cid in cluster_order:
    intra_cluster_schedule = nearest_neighbor_tsp(clusters[cid])
    final_schedule.extend(intra_cluster_schedule)

# -------------------------------
# Done! Print Final Calendar
# -------------------------------
print("\n🏁 Optimized F1 Calendar:")
for i, race in enumerate(final_schedule, 1):
    print(f"{i:02d}. {race}")



🏁 Optimized F1 Calendar:
01. Sakhir
02. Lusail
03. Yas Island
04. Jeddah
05. Baku
06. Imola
07. Monza
08. Monaco
09. Barcelona
10. Spa-Francorchamps
11. Zandvoort
12. Silverstone
13. Spielberg
14. Budapest
15. Miami
16. Austin
17. Mexico City
18. Las Vegas
19. Montréal
20. São Paulo
21. Melbourne
22. Marina Bay
23. Shanghai
24. Suzuka


In [9]:
len(track_names)

20

In [2]:
%pip install -q numpy scikit-learn
%pip install -q matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import random
import datetime

In [27]:
year = 2030
n = 1

random.seed(42)  # For reproducibility

# Step 1: Generate all Sundays in the year
sundays = []
dt = datetime.date(year, 1, 1)
while dt.year == year:
    if dt.weekday() == 6:  # Sunday
        sundays.append(dt)
    dt += datetime.timedelta(days=1)

In [154]:
def generate_calendar(year: int, n: int) -> list[str]:
    assert 2026 <= year <= 2030, "Year must be between 2026 and 2030"
    assert 15 <= n <= 25, "Races must be between 15 and 25"

    sundays = []
    dt = datetime.date(year, 1, 1)
    while dt.year == year:
        if dt.weekday() == 6:
            sundays.append(dt)
        dt += datetime.timedelta(days=1)

    season_start = max([d for d in sundays if d.month == 2])
    season_end = min([d for d in sundays if d.month == 12])
    sundays = [d for d in sundays if season_start <= d <= season_end]


    sundays = [d for d in sundays if not (d.month == 8 and d.day <= 21)]


    race_days = []
    triple_header_count = 0
    i = 0
    p = 1

    # Determine triple-header constraints
    if n == 25:
        triple_header = 3
    elif n > 21:
        triple_header = 2
    elif n > 19:
        triple_header = 1
    else:
        triple_header = 0

    while len(race_days) < n and i < len(sundays):
        current = sundays[i]

        # Avoid 4-in-a-row
        if len(race_days) >= 3:
            d1, d2, d3 = race_days[-3], race_days[-2], race_days[-1]
            if (
                (d2 - d1).days == 7 and
                (d3 - d2).days == 7 and
                (current - d3).days == 7
            ):
                i += 1
                continue  # skip to avoid 4-in-a-row
        # Insert a triple-header if we're at the beginning or near the end and still allowed
        if (
            triple_header_count < triple_header and
            len(race_days) + 3 <= n and
            i + 2 < len(sundays) and
            p == 0
        ):
            s1, s2, s3 = sundays[i], sundays[i + 1], sundays[i + 2]
            if (s2 - s1).days == 7 and (s3 - s2).days == 7:
                race_days.extend([s1, s2, s3])
                triple_header_count += 1
                i += 3
                p = triple_header_count+1 if triple_header == 3 else triple_header_count+6
                continue

        # Otherwise add this Sunday and the next one with a 2-week gap
        if not race_days or (current - race_days[-1]).days >= 14:
            race_days.append(current)
            if i + 1 < len(sundays):
                race_days.append(sundays[i + 1])
                if triple_header_count < 3:
                    p -= 1  # allow triple-header next time
                else:
                    p = 1
                i += 2
                

        i += 1

    # Truncate to n races
    race_days = race_days[:n]
    return [d.strftime("%d-%m") for d in race_days]

In [165]:
sundays = generate_calendar(2028, 23)

In [166]:
# Find all triplets of consecutive Sundays in the 'sundays' list
consecutive_triplets = [
    sundays[i:i+3]
    for i in range(len(sundays) - 2)
    if (datetime.datetime.strptime(sundays[i+1], "%d-%m").date() - datetime.datetime.strptime(sundays[i], "%d-%m").date()).days == 7 and
       (datetime.datetime.strptime(sundays[i+2], "%d-%m").date() - datetime.datetime.strptime(sundays[i+1], "%d-%m").date()).days == 7
]

# Count the number of triplets
len(consecutive_triplets)
consecutive_triplets

[['19-03', '26-03', '02-04'], ['01-10', '08-10', '15-10']]

In [167]:
len(sundays)

23

In [168]:
sundays

['27-02',
 '05-03',
 '19-03',
 '26-03',
 '02-04',
 '16-04',
 '23-04',
 '07-05',
 '14-05',
 '28-05',
 '04-06',
 '18-06',
 '25-06',
 '09-07',
 '16-07',
 '30-07',
 '27-08',
 '10-09',
 '17-09',
 '01-10',
 '08-10',
 '15-10',
 '29-10']

In [54]:
triple_headers=3

In [74]:
random.seed(23)  # For reproducibility

# Ensure there are enough Sundays to pick triple_headers triplets
if len(sundays) < triple_headers * 3:
    raise ValueError("Not enough Sundays to pick the required number of triplets.")

# Find all possible triplets of consecutive Sundays
triplets = [
    sundays[i:i+3]
    for i in range(len(sundays) - 2)
    if (sundays[i+1] - sundays[i]).days == 7 and (sundays[i+2] - sundays[i+1]).days == 7
]

# Randomly pick triple_headers triplets ensuring at least one Sunday from one triplet and another
selected_triplets = []
used_sundays = set()

while len(selected_triplets) < triple_headers:
    triplet = random.choice(triplets)
    if not any(sunday in used_sundays for sunday in triplet):
        selected_triplets.append(triplet)
        used_sundays.update(triplet)
        
        # Add min(date) - 7 days if it exists in sundays
        min_date_minus_7 = min(triplet) - datetime.timedelta(days=7)
        if min_date_minus_7 in sundays:
            used_sundays.add(min_date_minus_7)
        
        # Add max(date) + 7 days if it exists in sundays
        max_date_plus_7 = max(triplet) + datetime.timedelta(days=7)
        if max_date_plus_7 in sundays:
            used_sundays.add(max_date_plus_7)


print("Selected Triplets:")
for triplet in selected_triplets:
    print(triplet)

Selected Triplets:
[datetime.date(2028, 7, 2), datetime.date(2028, 7, 9), datetime.date(2028, 7, 16)]
[datetime.date(2028, 4, 2), datetime.date(2028, 4, 9), datetime.date(2028, 4, 16)]
[datetime.date(2028, 3, 5), datetime.date(2028, 3, 12), datetime.date(2028, 3, 19)]


In [56]:
n=25

In [75]:
final_calendar = []

# Add all Sundays from the selected triplets to the final calendar
for triplet in selected_triplets:
    final_calendar.extend(triplet)

# Ensure the used_sundays set is updated with the triplets
used_sundays.update(final_calendar)

# Randomly pick Sundays from the remaining pool until final_calendar contains n Sundays
remaining_pool = list(set(sundays) - used_sundays)
random.seed(23)  # For reproducibility

while len(final_calendar) < n:
    picked_sunday = random.choice(remaining_pool)
    final_calendar.append(picked_sunday)
    used_sundays.add(picked_sunday)
    remaining_pool.remove(picked_sunday)

    # Remove picked_sunday - 7 days if it exists in the remaining pool
    minus_7 = picked_sunday - datetime.timedelta(days=7)
    minus_14 = picked_sunday - datetime.timedelta(days=14)
    if minus_7 in remaining_pool and minus_14 not in remaining_pool:
        remaining_pool.remove(minus_7)

    # Remove picked_sunday + 7 days if it exists in the remaining pool
    plus_7 = picked_sunday + datetime.timedelta(days=7)
    plus_14 = picked_sunday + datetime.timedelta(days=14)
    if plus_7 in remaining_pool and plus_14 not in remaining_pool:
        remaining_pool.remove(plus_7)
        
    print(f"Picked Sunday: {picked_sunday}")
    print(f"Remaining Pool: {remaining_pool}")
    print(f"Final Calendar length: {len(final_calendar)}")

# Sort the final calendar for chronological order
final_calendar.sort()

print("Final Calendar:")
print(f"Total Sundays: {len(final_calendar)}")
for sunday in final_calendar:
    print(sunday)

Picked Sunday: 2028-05-07
Remaining Pool: [datetime.date(2028, 5, 21), datetime.date(2028, 11, 26), datetime.date(2028, 6, 18), datetime.date(2028, 6, 4), datetime.date(2028, 10, 1), datetime.date(2028, 9, 24), datetime.date(2028, 10, 8), datetime.date(2028, 10, 15), datetime.date(2028, 10, 29), datetime.date(2028, 5, 14), datetime.date(2028, 5, 28), datetime.date(2028, 6, 11), datetime.date(2028, 7, 30), datetime.date(2028, 9, 17), datetime.date(2028, 9, 3), datetime.date(2028, 11, 12), datetime.date(2028, 11, 5), datetime.date(2028, 10, 22), datetime.date(2028, 11, 19), datetime.date(2028, 8, 27), datetime.date(2028, 9, 10)]
Final Calendar length: 10
Picked Sunday: 2028-06-18
Remaining Pool: [datetime.date(2028, 5, 21), datetime.date(2028, 11, 26), datetime.date(2028, 6, 4), datetime.date(2028, 10, 1), datetime.date(2028, 9, 24), datetime.date(2028, 10, 8), datetime.date(2028, 10, 15), datetime.date(2028, 10, 29), datetime.date(2028, 5, 14), datetime.date(2028, 5, 28), datetime.date(

IndexError: Cannot choose from an empty sequence

Final Race Days:
2028-03-05
2028-03-12
2028-03-19
2028-04-02
2028-04-09
2028-04-16
2028-05-14
2028-05-21
2028-06-04
2028-06-18
2028-07-02
2028-07-09
2028-07-16
2028-07-30
2028-08-27
2028-09-10
2028-09-17
2028-09-24
2028-10-08
2028-10-15
2028-10-22
2028-10-29
2028-11-12
2028-11-19
2028-11-26


In [ ]:
complete the calendar in order to take

In [29]:
len(sundays)  # Number of Sundays after filtering

37

![alt text](image.png)

In [30]:
if n == 25:
    min_triple_header = 3
    max_triple_header = 3
elif n > 21:
    min_triple_header = 2
    max_triple_header = 2
elif n > 19:
    min_triple_header = 1
    max_triple_header = 2
else:
    min_triple_header = 0
    max_triple_header = 2
    

In [31]:
race_days = []
triple_header_count = 0

In [ ]:

# Step 3: Build the race calendar
race_days = []
triple_header_count = 0
max_triple_headers = 3

i = 0
while len(race_days) < n and i < len(sundays):
    if not race_days:
        race_days.append(sundays[i])
        i += 1
        continue

    last_race = race_days[-1]

    # Attempt to add triple-header
    if (
        triple_header_count < max_triple_headers and
        len(race_days) + 3 <= n and
        i + 2 < len(sundays)
    ):
        triple = sundays[i:i+3]

        # Ensure consecutive Sundays
        if (triple[1] - triple[0]).days == 7 and (triple[2] - triple[1]).days == 7:
            # Ensure this triple doesn’t create 4-in-a-row
            if len(race_days) >= 3:
                d1, d2, d3 = race_days[-3], race_days[-2], race_days[-1]
                if (d2 - d1).days == 7 and (d3 - d2).days == 7 and (triple[0] - d3).days == 7:
                    i += 1
                    continue  # would create 4-in-a-row

            race_days.extend(triple)
            triple_header_count += 1
            i += 3
            continue

    # Else try single race after a gap
    next_candidates = sundays[i:]
    for candidate in next_candidates:
        if candidate <= last_race:
            continue

        # Avoid 4-in-a-row
        if len(race_days) >= 3:
            d1, d2, d3 = race_days[-3], race_days[-2], race_days[-1]
            if (d2 - d1).days == 7 and (d3 - d2).days == 7 and (candidate - d3).days == 7:
                continue

        race_days.append(candidate)
        i = sundays.index(candidate) + 1
        break
    else:
        break  # No more valid candidates

# Truncate if necessary
race_days = race_days[:n]

AssertionError: Race count must be between 15 and 25